In [ ]:
import gym
import random
import os
import numpy as np
from collections      import deque
from tensorflow.keras.models     import Sequential
from tensorflow.keras.layers     import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
ENV_NAME = 'CartPole-v1'
model_dir = 'weights.h5'

In [ ]:
class Agent():
    def __init__(self, state_size, action_size):
        self.state_size         = state_size
        self.action_size        = action_size
        self.memory             = deque(maxlen=2000)
        self.learning_rate      = 0.001
        self.gamma              = 0.95
        self.exploration_rate   = 1.0
        self.exploration_min    = 0.01
        self.exploration_decay  = 0.995
        self.brain              = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(16, input_dim=self.state_size, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer="adam")

        return model
    
    def load_model(self, model_dir = None):
        model = self._build_model()
        model.load_weights(model_dir)
        print("Pretrained model loaded from:", model_dir)
        return model
    
    def save_model(self):
        self.brain.save('weights.h5')

    def act(self, state):
        if np.random.rand() <= self.exploration_rate:
            return random.randrange(self.action_size)
        act_values = self.brain.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, sample_batch_size):
        if len(self.memory) < sample_batch_size:
            return
        sample_batch = random.sample(self.memory, sample_batch_size)
        for state, action, reward, next_state, done in sample_batch:
            target = reward
            if not done:
              target = reward + self.gamma * np.amax(self.brain.predict(next_state, verbose=0)[0])
            target_f = self.brain.predict(state, verbose=0)
            target_f[0][action] = target
            self.brain.fit(state, target_f, epochs=1, verbose=0)
        if self.exploration_rate > self.exploration_min:
            self.exploration_rate *= self.exploration_decay

In [ ]:
train_env = gym.make(ENV_NAME)
state_size = train_env.observation_space.shape[0]
action_size = train_env.action_space.n
agent = Agent(state_size, action_size)
#agent.load_model(model_dir)
sample_batch_size = 32
TRAIN_EPISODES = 500
try:
    for i in range(TRAIN_EPISODES):
        state, info = train_env.reset()
        state = np.reshape(state, [1, state.shape[0]])
        done = False
        index = 0
        while not done:
            action = agent.act(state)
            next_state, reward, done, _, _ = train_env.step(action)
            next_state = np.reshape(next_state, [1, next_state.shape[0]])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            index += 1
        print("Episode {}# Score: {}".format(i, index + 1))
        agent.replay(sample_batch_size)
finally:
    agent.save_model()

In [ ]:
train_env = gym.make(ENV_NAME, render_mode = "human")
state_size = train_env.observation_space.shape[0]
action_size = train_env.action_space.n
agent = Agent(state_size, action_size)
agent.load_model(model_dir)
agent.exploration_rate = agent.exploration_min

sample_batch_size = 32
TEST_EPISODES = 100

while True:
    state, info = train_env.reset()
    state = np.reshape(state, [1, state.shape[0]])
    done = False
    index = 0
    while not done:
        action = agent.act(state)
        next_state, reward, done, _, _ = train_env.step(action)
        next_state = np.reshape(next_state, [1, next_state.shape[0]])
        state = next_state
        index += 1
    print("Episode {}# Score: {}".format(i, index + 1))